In [1]:
import os
import random
import numpy as np
from io import BytesIO

# Plotting and dealing with images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf

# Interactive widgets
from ipywidgets import widgets

In [2]:
BASE_DIR = r'C:\Users\toran\Documents\GitHub\ML_PadiCare\data'

train_dir = os.path.join(BASE_DIR, 'train')
test_dir = os.path.join(BASE_DIR, 'test')
validation_dir = os.path.join(BASE_DIR, 'val')
# Directory with training 
train_leafblight_dir = os.path.join(train_dir, 'bacterial_leaf_blight')
train_panicleblight_dir = os.path.join(train_dir, 'bacterial_panicle_blight')
train_brownspot_dir = os.path.join(train_dir, 'brown_spot')
train_sheathblight_dir = os.path.join(train_dir, 'rice_sheath_blight')
train_normal_dir = os.path.join(train_dir, 'normal')
# Directory with validation 
validation_leafblight_dir = os.path.join(validation_dir, 'bacterial_leaf_blight')
validation_panicleblight_dir = os.path.join(validation_dir, 'bacterial_panicle_blight')
validation_brownspot_dir = os.path.join(validation_dir, 'brown_spot')
validation_sheathblight_dir = os.path.join(validation_dir, 'rice_sheath_blight')
validation_normal_dir = os.path.join(validation_dir, 'normal')
# Directory with test 
test_leafblight_dir = os.path.join(test_dir, 'bacterial_leaf_blight')
test_panicleblight_dir = os.path.join(test_dir, 'bacterial_panicle_blight')
test_brownspot_dir = os.path.join(test_dir, 'brown_spot')
test_sheathblight_dir = os.path.join(test_dir, 'rice_sheath_blight')
test_normal_dir = os.path.join(test_dir, 'normal')
print(f"Contents of base directory: {os.listdir(BASE_DIR)}")
print(f"\nContents of train directory: {train_dir}")
print(f"\nContents of validation directory: {validation_dir}")
print(f"\nContents of test directory: {test_dir}")

Contents of base directory: ['test', 'train', 'val']

Contents of train directory: C:\Users\toran\Documents\GitHub\ML_PadiCare\data\train

Contents of validation directory: C:\Users\toran\Documents\GitHub\ML_PadiCare\data\val

Contents of test directory: C:\Users\toran\Documents\GitHub\ML_PadiCare\data\test


In [3]:
train_leafblight_fnames = os.listdir(train_leafblight_dir)
train_panicleblight_fnames = os.listdir(train_panicleblight_dir )
train_brownspot_fnames = os.listdir(train_brownspot_dir)
train_sheathblight_fnames = os.listdir(train_sheathblight_dir )
train_normal_fnames = os.listdir(train_normal_dir)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameter augmentasi
BATCH_SIZE = 20
IMG_HEIGHT, IMG_WIDTH = 150, 150

# Augmentasi data training
image_train_gen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.50,
    rotation_range=45,
    horizontal_flip=True,
    width_shift_range=0.15,
    height_shift_range=0.15
)

train_data_gen = image_train_gen.flow_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

# Normalisasi data validasi tanpa augmentasi
img_val_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = img_val_gen.flow_from_directory(
    validation_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

# Normalisasi data testing tanpa augmentasi
test_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = test_data_gen.flow_from_directory(
    test_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)


Found 1423 images belonging to 5 classes.
Found 604 images belonging to 5 classes.
Found 757 images belonging to 5 classes.


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),  # Dropout
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     9,470,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,496,613 (36.23 MB)

 Trainable params: 9,496,613 (36.23 MB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics = ['accuracy']
    )

In [7]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9
)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [67]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameter augmentasi
BATCH_SIZE = 20
IMG_HEIGHT, IMG_WIDTH = 150, 150

# Augmentasi data training
image_train_gen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.50,
    rotation_range=45,
    horizontal_flip=True,
    width_shift_range=0.15,
    height_shift_range=0.15
)

train_data_gen = image_train_gen.flow_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

# Normalisasi data validasi tanpa augmentasi
img_val_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = img_val_gen.flow_from_directory(
    validation_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)

# Normalisasi data testing tanpa augmentasi
test_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = test_data_gen.flow_from_directory(
    test_dir,
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical'
)


Found 1423 images belonging to 5 classes.
Found 604 images belonging to 5 classes.
Found 757 images belonging to 5 classes.


In [66]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = train_dataset.cache().shuffle(SHUFFLE_BUFFER_SIZE).prefetch(PREFETCH_BUFFER_SIZE)
validation_dataset_final = validation_dataset.cache().prefetch(PREFETCH_BUFFER_SIZE)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Pastikan pipeline `train_dataset` menggunakan generator yang sesuai
# Konversi ke array untuk menghitung class weights
train_labels = np.concatenate([y for _, y in train_data_gen])  # Ambil label dari generator

# Karena label dari `ImageDataGenerator` berbentuk kategori (class_mode='sparse'),
# tidak perlu menggunakan np.argmax
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Konversi class_weights ke format dictionary yang diterima oleh Keras
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

# Gunakan class_weights saat training
history = model.fit(
    train_dataset_final,  # Pipeline dengan augmentasi
    epochs=15,
    validation_data=validation_dataset_final,
    verbose=2,
    class_weight=class_weights
)


In [28]:
history = model.fit(
    train_dataset_final,
    epochs=15,
    validation_data=validation_dataset_final,
    verbose=2
    )

Epoch 1/15
115/115 - 37s - 321ms/step - accuracy: 0.9798 - loss: 0.1337 - val_accuracy: 0.8946 - val_loss: 0.5063
Epoch 2/15
115/115 - 26s - 228ms/step - accuracy: 0.9820 - loss: 0.1120 - val_accuracy: 0.8986 - val_loss: 0.5606
Epoch 3/15
115/115 - 28s - 248ms/step - accuracy: 0.9811 - loss: 0.1065 - val_accuracy: 0.8933 - val_loss: 0.4517
Epoch 4/15
115/115 - 33s - 283ms/step - accuracy: 0.9939 - loss: 0.0727 - val_accuracy: 0.8906 - val_loss: 0.5936
Epoch 5/15
115/115 - 33s - 288ms/step - accuracy: 0.9855 - loss: 0.0956 - val_accuracy: 0.8986 - val_loss: 0.4890
Epoch 6/15
115/115 - 33s - 289ms/step - accuracy: 0.9860 - loss: 0.0927 - val_accuracy: 0.8986 - val_loss: 0.5548
Epoch 7/15
115/115 - 33s - 288ms/step - accuracy: 0.9851 - loss: 0.0868 - val_accuracy: 0.9012 - val_loss: 0.4714
Epoch 8/15
115/115 - 28s - 243ms/step - accuracy: 0.9899 - loss: 0.0784 - val_accuracy: 0.9038 - val_loss: 0.5552
Epoch 9/15
115/115 - 28s - 247ms/step - accuracy: 0.9899 - loss: 0.0829 - val_accuracy: 

In [ ]:
# Definisikan widget FileUpload
uploader = widgets.FileUpload(accept="image/*", multiple=True)
display(uploader)

# Output widget untuk menampilkan hasil prediksi
out = widgets.Output()
display(out)

# Fungsi untuk memproses dan memprediksi setiap file gambar
def file_predict(filename, file, out):
    """Fungsi untuk membuat prediksi dan mencetak hasilnya."""
    # Load gambar, ubah ukuran, dan ubah ke array
    image = tf.keras.utils.load_img(file, target_size=(150, 150))
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalisasi

    # Prediksi menggunakan model
    predictions = model.predict(image, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Mapping label kelas
    class_names = [
        "Bacterial Leaf Blight",
        "Bacterial Panicle Blight",
        "Brown Spot",
        "Rice Sheath Blight",
        "Normal"
    ]
    predicted_label = class_names[predicted_class]

    # Tampilkan hasil prediksi
    with out:
        print(f"{filename} -> {predicted_label}")

# Fungsi untuk menangani perubahan pada uploader
def on_upload_change(change):
    """Fungsi untuk menangani file yang diunggah dan memproses prediksi."""
    # Bersihkan output sebelumnya
    out.clear_output()

    # Ambil file yang baru diunggah
    items = change.new
    for item in items:  # Loop jika ada lebih dari satu file
        file_jpgdata = BytesIO(item.content)
        file_predict(item.name, file_jpgdata, out)

# Hubungkan fungsi observasi ke uploader
uploader.observe(on_upload_change, names='value')


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

Output()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])